In [ ]:
# 해당 크롤링은 colab 대신에 vs_code로 수행함.

In [ ]:
pip install pandas

In [ ]:
pip install selenium

In [ ]:
pip install bs4

In [ ]:
pip install webdriver_manager

In [ ]:
pip install tqdm

In [ ]:
pip install lxml

In [ ]:
pip install openpyxl

In [ ]:
pip install ipywidgets

In [ ]:
pip install --upgrade jupyter ipywidgets

In [ ]:
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver # 브라우저를 조작하는 모듈
from selenium.webdriver.chrome.service import Service # 크롬 드라이버의 시작과 중지를 담당하는 클래스
from selenium.webdriver.common.by import By # 엘리먼트를 어떤 방식으로 선택할 지에 대한 상수
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 관리
import pandas as pd
from tqdm.auto import tqdm
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

#### 저는 명시적 대기를 사용하였지만 암묵적 대기를 사용하시면 더 빠르게 실행되게 할 수도 있습니다.

In [ ]:
DATA_PATH = "C:/Users/"

In [ ]:
def influence_searching(ID):
    url = "https://www.instagram.com/" + str(ID)
    return url

In [ ]:
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, "div._aagw")[0]
    first.click()
    time.sleep(3)

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 본문 내용
    try:
        content = soup.select('div._a9zs')[0].text
    except:
        content = ''
    # 해시 태그
    tags = re.findall(r'#[^\s#,\\]+', content)
    # 작성 일자
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    # 좋아요
    try:
        like = soup.select('section._ae5m._ae5n._ae5o > div > div > span > a > span')[0].text
        like = re.findall(r'[0-9]+', like)[0]
    except:
        like = 0

    # 댓글 더보기 클릭
    action = ActionChains(driver)
    for _ in range(100):
        try:
            next_coment = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button')
            action.move_to_element(next_coment).perform()
            driver.implicitly_wait(2)
            # time.sleep(1.5)
            next_coment.click()
            driver.implicitly_wait(1)
            # time.sleep(0.5)
        except:
            break
    # 아이디 및 댓글 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h3._a9zc")))
    try:
        username = [username.div.div.div.a.get_text() for username in soup.find_all('h3', {'class' :'_a9zc'})]
        comments = [coment.get_text() for coment in soup.find_all('span', {'class':'_aacl _aaco _aacu _aacx _aad7 _aade'})]
    except:
        username = ''
        comments = ''

    content = re.sub(r'#[^\s#,\\]+', "", content)
    data = [content, username, comments, date, like, tags]
    return data

In [ ]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh')
    right.click()
    time.sleep(1.5)

In [ ]:
# 인스타그램에 로그인
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get('https://www.instagram.com')
time.sleep(3)

email = str(input('아이디:')) # 아이디 입력
input_id = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(1) > div > label > input').send_keys(email)

password = str(input('비밀번호:')) # 페스워드 입력
input_pw = driver.find_element(By.CSS_SELECTOR, '#loginForm > div > div:nth-child(2) > div > label > input').send_keys(password)
time.sleep(1)

driver.find_element(By.CSS_SELECTOR, "#loginForm > div > div:nth-child(3)").click()

In [ ]:
df = pd.read_excel(f"{DATA_PATH}인플루언서 리스트.xlsx")
id_list = df['ID'].iloc[:]

id_list

In [ ]:
# 한명의 인플루언서마다 게시글 100개 크롤링
target = 100
for infuluence in tqdm(id_list, desc="outer", position=0):
    driver.get(f'https://www.instagram.com/{infuluence}/')

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span._ac2a")))
    count = int(driver.find_element(By.CSS_SELECTOR, "span._ac2a").text)

    driver.implicitly_wait(10)

    select_first(driver)
    results = []
    for j in tqdm(range(target), desc="inner", position=1):
        driver.implicitly_wait(10)
        if j >= count:
            break
        try:
            data = get_content(driver)
            results.append(data)
            move_next(driver)
        except:
            time.sleep(2)
            move_next(driver)

        time.sleep(3)

    results_df = pd.DataFrame(results)
    results_df.columns = ['content','username', 'comments', 'date', 'like', 'tags']
    results_df.to_csv(f'{DATA_PATH}_{infuluence}_{count}insta_crawling.csv', encoding='utf-8-sig')
    # 다음 ID로 넘어가기전 너무 빠르게 하면 오류 및 사람인지 확인하는 메시지가 떠서 저는 우선 30초를 줬습니다.
    # 본인이 해보시고 오류가 안나는 선에서 시간을 정하시면 됩니다.

    time.sleep(30)
driver.quit()